In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR
from matplotlib import pyplot as plt
from tools import *
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import PolynomialFeatures

In [2]:
def getObs(n):
    obs = {}
    for i in range(n):
        try:
            df = pd.read_csv(f'rocket-results/{i}.csv')
        except:
            print('Missing', i)
        obs[i] = df
    return obs

In [3]:
def initializeConfig():
    configs = {}
    with open('sample_list.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_100.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_200.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_300.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    return configs

In [4]:
config = initializeConfig()
obs = getObs(400)

In [5]:
data = {}
for i in range(len(config)):
    data[i] = list(config[i]['S'].values()) + list(config[i]['B'].values())
    data[i].append(obs[i].max()['Altitude (ft)'])
    data[i].append(obs[i].mean()['Stability Margin (cal)'])
    data[i].append(obs[i].max()['Time (sec)'])
df = pd.DataFrame.from_dict(data, orient='index')
df

/var/folders/ff/y3gj2gld0zqfjtq_lhxz03t00000gn/T/ipykernel_26772/1048845639.py:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data[i].append(obs[i].mean()['Stability Margin (cal)'])


,0,1,2,3,4,5,6,7,8,9,10
0,9.465233,9.191411,6.313146,6.181568,4.253219,3.362766,2.004250,3.767645,0.004903,-5.121752,0.0100
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,77117.160000,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,51216.000000,5.212006,125.0071
3,5.590219,9.233072,6.352605,8.429602,5.672564,5.705322,5.717624,3.420081,0.004691,-2.489141,0.0100
4,2.916229,4.216994,4.711545,6.462613,4.887217,2.638681,9.605809,5.234709,0.004818,-4.231226,0.0100
...,...,...,...,...,...,...,...,...,...,...,...
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,70830.750000,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,72485.310000,1.614970,150.9995
397,7.364198,8.455093,4.617628,3.805883,4.280772,6.398586,8.552977,8.138172,0.000000,-1.734951,0.0000
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,73446.170000,0.853824,150.9995


In [6]:
df = df.rename(columns={0: "Schord", 1: "Sspan", 2: "Ssweep", 3: "Stip", 4: "Bchord", 
                        5: "Bspan", 6: "Bsweep", 7: "Btip", 8: "Altitude", 9: "Stability", 10: "Time"})
df['Schordspan'] = df['Schord'] * df['Sspan']
df['Schordsweep'] = df['Schord'] * df['Ssweep']
df['Schordtip'] = df['Schord'] * df['Stip']
df['Sspansweep'] = df['Sspan'] * df['Ssweep']
df['SSpantip'] = df['Sspan'] * df['Stip']
df['SSweeptip'] = df['Ssweep'] * df['Stip']

df['Bchordspan'] = df['Bchord'] * df['Bspan']
df['Bchordsweep'] = df['Bchord'] * df['Bsweep']
df['Bchordtip'] = df['Bchord'] * df['Btip']
df['Bspansweep'] = df['Bspan'] * df['Bsweep']
df['BSpantip'] = df['Bspan'] * df['Btip']
df['BSweeptip'] = df['Bsweep'] * df['Btip']
alt = df.pop("Altitude")
stab = df.pop("Stability")
time = df.pop("Time")

df.insert(len(df.columns), "Altitude", alt)
df.insert(len(df.columns), "Stability", stab)
df.insert(len(df.columns), "Time", time)
df

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
0,9.465233,9.191411,6.313146,6.181568,4.253219,3.362766,2.004250,3.767645,86.998844,59.755399,...,39.025144,14.302581,8.524515,16.024619,6.739826,12.669711,7.551304,0.004903,-5.121752,0.0100
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.160000,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.000000,5.212006,125.0071
3,5.590219,9.233072,6.352605,8.429602,5.672564,5.705322,5.717624,3.420081,51.614896,35.512459,...,53.549937,32.363803,32.433584,19.400628,32.620884,19.512664,19.554736,0.004691,-2.489141,0.0100
4,2.916229,4.216994,4.711545,6.462613,4.887217,2.638681,9.605809,5.234709,12.297724,13.739946,...,30.448892,12.895807,46.945675,25.583158,25.346667,13.812727,50.283615,0.004818,-4.231226,0.0100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.750000,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.310000,1.614970,150.9995
397,7.364198,8.455093,4.617628,3.805883,4.280772,6.398586,8.552977,8.138172,62.264979,34.005123,...,17.574150,27.390888,36.613344,34.837657,54.726959,52.072793,69.605594,0.000000,-1.734951,0.0000
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.170000,0.853824,150.9995


In [7]:
import torch
from torch.utils import data
from torch import nn

In [8]:
def synthetic_data(m, c, num_examples):
 
    """Generate y = mX + bias(c) + noise"""

    X = torch.normal(0, 1, (num_examples, len(m)))

    y = torch.matmul(X, m) + c

    y += torch.normal(0, 0.01, y.shape)
 
    return X, y.reshape((-1, 1))

In [9]:
synthetic_data(torch.Tensor([4,2]),2,10)

(tensor([[ 1.2854,  0.7723],
         [-0.2868,  0.3979],
         [ 1.1170, -0.8443],
         [ 0.6094,  1.2585],
         [-1.4640, -0.7778],
         [-1.6313, -1.2966],
         [-0.1940, -0.8472],
         [ 0.0800,  1.1407],
         [-1.0706, -0.9298],
         [-1.4800,  1.0732]]),
 tensor([[ 8.6980],
         [ 1.6593],
         [ 4.7598],
         [ 6.9457],
         [-5.3984],
         [-7.1112],
         [-0.4769],
         [ 4.6005],
         [-4.1380],
         [-1.7762]]))

In [10]:
filtered_df = df[df['Time'] > 90]
filtered_df

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.16,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.00,5.212006,125.0071
5,8.562160,4.337768,7.930397,4.372560,9.417980,8.246008,2.728282,6.326085,37.140668,67.901332,...,34.676133,77.660732,25.694900,59.578937,22.497430,52.164942,17.259340,54082.50,4.485508,129.0079
6,8.703811,2.188438,9.143090,2.635864,9.087634,8.094306,6.826608,3.754130,19.047753,79.579730,...,24.099946,73.558093,62.037714,34.116158,55.256655,30.387077,25.627973,78784.51,0.613785,156.9962
8,8.074672,3.479253,5.061755,2.201202,8.430403,6.003352,2.080965,3.394916,28.093824,40.872014,...,11.141944,50.610675,17.543371,28.620512,12.492762,20.380874,7.064700,59576.45,2.920013,136.0078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,5.247843,7.799644,8.441005,9.658211,6.937770,7.882306,6.353957,9.196173,40.931306,44.297067,...,81.524999,54.685626,44.082290,63.800932,50.083829,72.487046,58.432082,39893.78,5.851534,111.0041
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.75,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.31,1.614970,150.9995
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.17,0.853824,150.9995


In [11]:
poly = PolynomialFeatures(2)
raw_X = filtered_df.iloc[:,:8]
raw_Y = filtered_df.iloc[:,-3]
batch_size = 10

In [12]:
train_X = torch.from_numpy(poly.fit_transform(raw_X[:100])).double()
train_Y = torch.from_numpy(np.array([[x] for x in raw_Y[:100]])).double()

test_X = torch.from_numpy(poly.fit_transform(raw_X[100:])).double()
test_Y = torch.from_numpy(np.array([[x] for x in raw_Y[100:]])).double()

/var/folders/ff/y3gj2gld0zqfjtq_lhxz03t00000gn/T/ipykernel_26772/2555709260.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  train_Y = torch.from_numpy(np.array([[x] for x in raw_Y[:100]])).double()
/var/folders/ff/y3gj2gld0zqfjtq_lhxz03t00000gn/T/ipykernel_26772/2555709260.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  test_Y = torch.from_numpy(np.array([[x] for x in raw_Y[100:]])).double()


In [13]:
def load_arrays(data_arrays, batch_size, train = True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle = train)

In [14]:
data_iter = load_arrays((train_X, train_Y), 10)

In [15]:
data_iter

In [16]:
class Net(torch.nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.linear1 = torch.nn.Linear(45, 90)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(90, 60)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(60, 30)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(30, 10)
        self.activation4 = torch.nn.ReLU()
        self.linear5 = torch.nn.Linear(10, 1)

    def forward(self, x):
        
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.linear4(x)
        x = self.activation4(x)
        x = self.linear5(x)
        
        return 1000000*x

In [77]:
net = Net()
net.double()
net(train_X)

tensor([[1566936.7151],
        [2274566.4782],
        [2017070.0403],
        [2039873.2304],
        [1138180.6992],
        [2666942.8408],
        [1735451.2055],
        [2522809.9438],
        [2311030.9688],
        [1953926.9801],
        [2096791.0447],
        [1596315.3750],
        [1029697.0929],
        [1647268.8905],
        [1988854.2510],
        [1808579.6445],
        [2573164.4724],
        [2249685.1552],
        [1811350.1735],
        [1157661.8302],
        [ 896379.8506],
        [1464640.5811],
        [1326238.9880],
        [1637364.0241],
        [1352638.6554],
        [1008419.1327],
        [1852160.2152],
        [1155330.0052],
        [2422272.4514],
        [2549873.0814],
        [2432473.7201],
        [1672326.8915],
        [2063084.2377],
        [1258311.2397],
        [1577840.2720],
        [ 755757.5469],
        [1234454.0214],
        [2894834.7077],
        [1397544.8623],
        [2612507.2520],
        [1148435.4720],
        [1342944

In [78]:
loss = nn.MSELoss()

In [79]:
trainer = torch.optim.SGD(net.parameters(), lr= 0.0000000000000000001) # 0.000001, 0.0000000000000001 00000000000000001
net(train_X)

tensor([[1566936.7151],
        [2274566.4782],
        [2017070.0403],
        [2039873.2304],
        [1138180.6992],
        [2666942.8408],
        [1735451.2055],
        [2522809.9438],
        [2311030.9688],
        [1953926.9801],
        [2096791.0447],
        [1596315.3750],
        [1029697.0929],
        [1647268.8905],
        [1988854.2510],
        [1808579.6445],
        [2573164.4724],
        [2249685.1552],
        [1811350.1735],
        [1157661.8302],
        [ 896379.8506],
        [1464640.5811],
        [1326238.9880],
        [1637364.0241],
        [1352638.6554],
        [1008419.1327],
        [1852160.2152],
        [1155330.0052],
        [2422272.4514],
        [2549873.0814],
        [2432473.7201],
        [1672326.8915],
        [2063084.2377],
        [1258311.2397],
        [1577840.2720],
        [ 755757.5469],
        [1234454.0214],
        [2894834.7077],
        [1397544.8623],
        [2612507.2520],
        [1148435.4720],
        [1342944

In [80]:
num_epochs = 1000
 
for epoch in range(num_epochs):
 
    for X, y in data_iter:

        l = loss(net(X) , 1000000*y)

        trainer.zero_grad() #sets gradients to zero

        l.backward() # back propagation

        trainer.step() # parameter update

    l = loss(net(train_X), train_Y)

    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 139827123015.242493
epoch 2, loss 39423904381.664055
epoch 3, loss 225605133.774900
epoch 4, loss 50556786487.609123
epoch 5, loss 247024843114.840668
epoch 6, loss 748533640242.335205
epoch 7, loss 2032978792176.225586
epoch 8, loss 5934110020903.773438
epoch 9, loss 2919961241092301.000000
epoch 10, loss 197567834109.109131
epoch 11, loss 308855106184.868591
epoch 12, loss 444558119146.787354
epoch 13, loss 597455360495.139404
epoch 14, loss 772918043874.622559
epoch 15, loss 970946039950.917480
epoch 16, loss 1191539213000.970459
epoch 17, loss 1434697429113.050537
epoch 18, loss 1700420551406.716553
epoch 19, loss 1988708452239.810547
epoch 20, loss 2299560984570.466797
epoch 21, loss 2632978027863.569824
epoch 22, loss 2988959428155.395508
epoch 23, loss 3367505055334.310547
epoch 24, loss 3768614767669.550781
epoch 25, loss 4192288450459.741211
epoch 26, loss 4638525949487.864258
epoch 27, loss 5107327143831.946289
epoch 28, loss 5598691890247.082031
epoch 29, loss 

epoch 244, loss 640302391297596.125000
epoch 245, loss 645686845828272.500000
epoch 246, loss 651093834368717.875000
epoch 247, loss 656523356889615.000000
epoch 248, loss 661975413134833.375000
epoch 249, loss 667450003133601.250000
epoch 250, loss 672947126411766.500000
epoch 251, loss 678466783161873.750000
epoch 252, loss 684008973070844.750000
epoch 253, loss 689573696090736.375000
epoch 254, loss 695160952428126.375000
epoch 255, loss 700770741261315.250000
epoch 256, loss 706403062991775.250000
epoch 257, loss 712057917066321.875000
epoch 258, loss 717735303869677.375000
epoch 259, loss 723435222660196.000000
epoch 260, loss 729157673734271.250000
epoch 261, loss 734902656952544.750000
epoch 262, loss 740670171945780.125000
epoch 263, loss 746460218792434.750000
epoch 264, loss 752272797069180.500000
epoch 265, loss 758107907217342.500000
epoch 266, loss 763965548405923.500000
epoch 267, loss 769845720817317.875000
epoch 268, loss 775748424262975.250000
epoch 269, loss 781673658

epoch 501, loss 2765009367614385.500000
epoch 502, loss 2776180626488079.500000
epoch 503, loss 2787374384874160.500000
epoch 504, loss 2798590642073861.000000
epoch 505, loss 2809829398620149.500000
epoch 506, loss 2821090653962644.000000
epoch 507, loss 2832374408526866.000000
epoch 508, loss 2843680660932860.000000
epoch 509, loss 2855009412286959.500000
epoch 510, loss 2866360661896798.500000
epoch 511, loss 2877734409895244.500000
epoch 512, loss 2889130655748676.000000
epoch 513, loss 2900549399773234.500000
epoch 514, loss 2911990641795797.000000
epoch 515, loss 2923454381370756.000000
epoch 516, loss 2934940618764234.000000
epoch 517, loss 2946449353420466.000000
epoch 518, loss 2957980585261154.500000
epoch 519, loss 2969534314320726.500000
epoch 520, loss 2981110540358222.000000
epoch 521, loss 2992709263199506.000000
epoch 522, loss 3004330482628981.000000
epoch 523, loss 3015974198766639.500000
epoch 524, loss 3027640411767589.000000
epoch 525, loss 3039329121006698.000000


epoch 709, loss 5572796861256486.000000
epoch 710, loss 5588645070869524.000000
epoch 711, loss 5604515751749888.000000
epoch 712, loss 5620408904013013.000000
epoch 713, loss 5636324526587001.000000
epoch 714, loss 5652262620386443.000000
epoch 715, loss 5668223184917395.000000
epoch 716, loss 5684206220104699.000000
epoch 717, loss 5700211725060312.000000
epoch 718, loss 5716239700378194.000000
epoch 719, loss 5732290146079698.000000
epoch 720, loss 5748363061948422.000000
epoch 721, loss 5764458447554498.000000
epoch 722, loss 5780576302794563.000000
epoch 723, loss 5796716627714294.000000
epoch 724, loss 5812879422010808.000000
epoch 725, loss 5829064685670221.000000
epoch 726, loss 5845272418774284.000000
epoch 727, loss 5861502620821757.000000
epoch 728, loss 5877755292301292.000000
epoch 729, loss 5894030431954665.000000
epoch 730, loss 5910328040367551.000000
epoch 731, loss 5926648117322527.000000
epoch 732, loss 5942990662428740.000000
epoch 733, loss 5959355676026130.000000


epoch 916, loss 9332288787313744.000000
epoch 917, loss 9352785665270516.000000
epoch 918, loss 9373304987140392.000000
epoch 919, loss 9393846751724188.000000
epoch 920, loss 9414410959414810.000000
epoch 921, loss 9434997609804840.000000
epoch 922, loss 9455606702582182.000000
epoch 923, loss 9476238238815928.000000
epoch 924, loss 9496892217141604.000000
epoch 925, loss 9517568637666218.000000
epoch 926, loss 9538267501423130.000000
epoch 927, loss 9558988806790950.000000
epoch 928, loss 9579732553499660.000000
epoch 929, loss 9600498742128664.000000
epoch 930, loss 9621287372744420.000000
epoch 931, loss 9642098445216202.000000
epoch 932, loss 9662931958547928.000000
epoch 933, loss 9683787913627848.000000
epoch 934, loss 9704666309469696.000000
epoch 935, loss 9725567146199662.000000
epoch 936, loss 9746490423853830.000000
epoch 937, loss 9767436142197080.000000
epoch 938, loss 9788404301313668.000000
epoch 939, loss 9809394900745990.000000
epoch 940, loss 9830407940812106.000000


In [81]:
np.sqrt(l.detach().numpy()/100)

10550948.333813772

In [82]:
np.sqrt((loss(net(test_X), test_Y)/100).detach().numpy())

10550798.238641398

In [83]:
# NN: weird landscape. Need to start with less stepsize (e.g. 0.000001) and then grow to 0.01

In [84]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()

        self.linear1 = torch.nn.Linear(45, 90)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(90, 60)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(60, 30)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(30, 10)
        self.activation4 = torch.nn.ReLU()
        self.linear5 = torch.nn.Linear(10, 1)

    def forward(self, x):
        
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.linear4(x)
        x = self.activation4(x)
        x = self.linear5(x)
        
        return torch.sigmoid(x)

In [85]:
classif = Classifier()
classif.double()

Classifier(
  (linear1): Linear(in_features=45, out_features=90, bias=True)
  (activation1): ReLU()
  (linear2): Linear(in_features=90, out_features=60, bias=True)
  (activation2): ReLU()
  (linear3): Linear(in_features=60, out_features=30, bias=True)
  (activation3): ReLU()
  (linear4): Linear(in_features=30, out_features=10, bias=True)
  (activation4): ReLU()
  (linear5): Linear(in_features=10, out_features=1, bias=True)
)

In [86]:
poly = PolynomialFeatures(2)
raw_X = df.iloc[:,:8]
raw_Y = df.iloc[:,-1] > 90
batch_size = 10

train_X = torch.from_numpy(poly.fit_transform(raw_X[:300])).double()
train_Y = torch.from_numpy(np.array([[x] for x in raw_Y[:300]])).double()

test_X = torch.from_numpy(poly.fit_transform(raw_X[300:])).double()
test_Y = torch.from_numpy(np.array([[x] for x in raw_Y[300:]])).double()
data_iter = load_arrays((train_X, train_Y), 10)

In [87]:
classif(train_X)

tensor([[0.2999],
        [0.3276],
        [0.2004],
        [0.2773],
        [0.3127],
        [0.2808],
        [0.2493],
        [0.3111],
        [0.3474],
        [0.2254],
        [0.2568],
        [0.2449],
        [0.2603],
        [0.3927],
        [0.1509],
        [0.2647],
        [0.2353],
        [0.2110],
        [0.2310],
        [0.2591],
        [0.2586],
        [0.2787],
        [0.2922],
        [0.3526],
        [0.3100],
        [0.3289],
        [0.3177],
        [0.3005],
        [0.3636],
        [0.3315],
        [0.2637],
        [0.2690],
        [0.2666],
        [0.2947],
        [0.1991],
        [0.2061],
        [0.2883],
        [0.3119],
        [0.3452],
        [0.2944],
        [0.3350],
        [0.2982],
        [0.3994],
        [0.2790],
        [0.2979],
        [0.2751],
        [0.3520],
        [0.2998],
        [0.2388],
        [0.3068],
        [0.2633],
        [0.2935],
        [0.3250],
        [0.2560],
        [0.3561],
        [0

In [88]:
loss = nn.BCELoss()

In [90]:
trainer = torch.optim.SGD(classif.parameters(), lr=0.01)
num_epochs = 250
for epoch in range(num_epochs):
 
    for X, y in data_iter:
        X.requires_grad = True
        l = loss(classif(X) ,y)

        trainer.zero_grad() #sets gradients to zero

        l.backward() # back propagation

        trainer.step() # parameter update
        
    l = loss(classif(train_X), train_Y)
    if epoch % 50 == 0:
        print(f'epoch {epoch + 1}, loss {l:f}')
        #print(X.grad)

epoch 1, loss 0.000382
epoch 51, loss 0.000266
epoch 101, loss 0.000203
epoch 151, loss 0.000162
epoch 201, loss 0.000134


In [91]:
loss(classif(test_X), test_Y)

tensor(1.5019, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)

In [92]:
sum((classif(test_X) >= 0.5) == test_Y)/len(test_Y)

tensor([0.8300])

In [ ]:
def coordinateAscentNN(stab_classifier, alt_predictor, initial = [0,0,0,0,0,0,0,0], inc = 0.0000001, max_iter = 10000, confidence = 0.95):
    '''
    Keep increasing in direction until unstable. 
    '''
    x = initial
    old = qr.predict(poly.fit_transform([x]))
    for _ in range(max_iter):
        #for i in range(8):
        y = np.array(x)
        # y[i] += np.sign(alt_predictor.coef_[i])*inc
        # print(y, partials(x)*inc)
        y += partials(x) * inc
        if np.all(y > 2.5) and qr.predict(poly.fit_transform([y])) > old:
            nxt = stab_classifier.predict_proba(poly.fit_transform([y]))[0,1]
            if nxt > confidence or np.random.rand() > 0.5:
                if nxt <= confidence:
                    print('Non greedy step')
                x = y[:]
                #print(qr.predict(poly.fit_transform([y])) - old)
                old = qr.predict(poly.fit_transform([y]))
                continue
            else:
                break
        
        print('No more feasible directions at iteration', _, '.', qr.predict(poly.fit_transform([x])) - qr.predict(poly.fit_transform([initial])))
        break
    return x

In [ ]:
def transform(x):
    L = [torch.Tensor([1]), x]
    for i in range(len(x)):
        for j in range(i, len(x)):
            L.append(torch.Tensor([x[i]*x[j]]))
    return torch.cat(L).double()

In [ ]:
transform(x)

In [ ]:
x = torch.Tensor([3,10,6,7,9,2.1,5,2])
# x.requires_grad = True
z = transform(x)
z.requires_grad = True
out = net(z)
#torch.autograd.grad(outputs = out, inputs = x, retain_graph = True)
x.grad

In [ ]:
out.backward()

In [ ]:
z.grad.data

In [ ]:
out[0]

In [ ]:
x